# 01 - Analyse Exploratoire des Données (EDA)

**Objectifs :**
1. Comprendre le périmètre et la qualité des données
2. Calculer les KPIs business (justifier le besoin de segmentation)
3. Explorer les features RFM et décider des transformations
4. Documenter les décisions pour le preprocessing

**Dataset :** UCI Online Retail (541 909 transactions, 8 variables)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

%matplotlib inline

In [ ]:
# Chemins robustes
ROOT = Path.cwd()
if not (ROOT / "data" / "raw" / "Online Retail.xlsx").exists():
    ROOT = ROOT.parent

DATA_PATH = ROOT / "data" / "raw" / "Online Retail.xlsx"
OUTPUT_DIR = ROOT / "outputs" / "figures"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"ROOT: {ROOT}")
print(f"DATA_PATH exists: {DATA_PATH.exists()}")

---
## 1. Chargement et contexte

In [ ]:
# Chargement
df = pd.read_excel(DATA_PATH)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

print(f"Shape: {df.shape}")
print(f"Mémoire: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

In [ ]:
# Aperçu
df.head()

In [ ]:
# Tableau récapitulatif contexte
context = {
    "Période": f"{df['InvoiceDate'].min().date()} → {df['InvoiceDate'].max().date()}",
    "Durée (jours)": (df['InvoiceDate'].max() - df['InvoiceDate'].min()).days,
    "Transactions": f"{len(df):,}",
    "Factures uniques": f"{df['InvoiceNo'].nunique():,}",
    "Clients uniques": f"{df['CustomerID'].nunique():,}",
    "Produits uniques": f"{df['StockCode'].nunique():,}",
    "Pays": df['Country'].nunique(),
    "Pays principal": f"{df['Country'].value_counts().index[0]} ({df['Country'].value_counts().iloc[0]/len(df)*100:.1f}%)"
}
pd.DataFrame.from_dict(context, orient='index', columns=['Valeur'])

---
## 2. Qualité des données (décisions nettoyage)

In [ ]:
# Valeurs manquantes
missing = pd.DataFrame({
    'Manquants': df.isnull().sum(),
    '%': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('%', ascending=False)
missing[missing['Manquants'] > 0]

In [ ]:
# Anomalies
anomalies = {
    "CustomerID null": (df['CustomerID'].isnull().sum(), df['CustomerID'].isnull().mean() * 100),
    "Quantity <= 0": ((df['Quantity'] <= 0).sum(), (df['Quantity'] <= 0).mean() * 100),
    "UnitPrice <= 0": ((df['UnitPrice'] <= 0).sum(), (df['UnitPrice'] <= 0).mean() * 100),
    "Factures 'C' (annulées)": (df['InvoiceNo'].astype(str).str.startswith('C').sum(), 
                                 df['InvoiceNo'].astype(str).str.startswith('C').mean() * 100)
}
pd.DataFrame.from_dict(anomalies, orient='index', columns=['Count', '%'])

In [ ]:
# Visualisation valeurs manquantes
fig, ax = plt.subplots(figsize=(8, 3))
missing_plot = missing[missing['Manquants'] > 0]
ax.barh(missing_plot.index, missing_plot['%'], color='coral')
ax.set_xlabel('% manquant')
ax.set_title('Valeurs manquantes')
for i, v in enumerate(missing_plot['%']):
    ax.text(v + 0.5, i, f'{v:.1f}%', va='center')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "01_missing_values.png", dpi=200, bbox_inches="tight")
plt.show()

### DÉCISIONS NETTOYAGE

| Anomalie | Action | Justification |
|----------|--------|---------------|
| CustomerID null (~25%) | **Supprimer** | Obligatoire pour RFM |
| Quantity ≤ 0 (~2%) | **Exclure** | Retours/annulations |
| UnitPrice ≤ 0 (~0.4%) | **Exclure** | Prix invalides |
| Factures 'C' | **Exclure** | Annulations explicites |

---
## 3. KPIs Business

**Objectif** : Quantifier la valeur et justifier le besoin de segmentation

In [ ]:
# Données valides pour KPIs (hors anomalies)
df_valid = df[
    (df['CustomerID'].notna()) & 
    (df['Quantity'] > 0) & 
    (df['UnitPrice'] > 0) &
    (~df['InvoiceNo'].astype(str).str.startswith('C'))  # Exclure annulations
].copy()

df_valid['TotalAmount'] = df_valid['Quantity'] * df_valid['UnitPrice']
print(f"Transactions valides: {len(df_valid):,} ({len(df_valid)/len(df)*100:.1f}%)")

In [ ]:
# KPIs globaux
kpis = {
    "Revenue total (£)": f"{df_valid['TotalAmount'].sum():,.0f}",
    "Nb commandes": f"{df_valid['InvoiceNo'].nunique():,}",
    "Nb clients": f"{df_valid['CustomerID'].nunique():,}",
    "AOV (Average Order Value)": f"£{df_valid.groupby('InvoiceNo')['TotalAmount'].sum().mean():,.2f}",
    "Panier moyen par ligne": f"£{df_valid['TotalAmount'].mean():,.2f}",
    "Revenue/client moyen": f"£{df_valid.groupby('CustomerID')['TotalAmount'].sum().mean():,.2f}"
}
pd.DataFrame.from_dict(kpis, orient='index', columns=['Valeur'])

In [ ]:
# Revenue par pays (Top 5)
revenue_country = df_valid.groupby('Country')['TotalAmount'].sum().sort_values(ascending=False)
top5 = revenue_country.head(5)
top5_pct = (top5 / revenue_country.sum() * 100).round(1)

print("Top 5 pays par revenue:")
pd.DataFrame({'Revenue (£)': top5.apply(lambda x: f"{x:,.0f}"), '% total': top5_pct})

In [ ]:
# PARETO : Concentration de valeur
customer_revenue = df_valid.groupby('CustomerID')['TotalAmount'].sum().sort_values(ascending=False)
customer_revenue_cumsum = customer_revenue.cumsum() / customer_revenue.sum() * 100

# Trouver le % de clients pour 80% du CA
n_customers = len(customer_revenue)
for pct in [10, 20, 30]:
    n_top = int(n_customers * pct / 100)
    revenue_pct = customer_revenue.head(n_top).sum() / customer_revenue.sum() * 100
    print(f"Top {pct}% clients ({n_top:,}) = {revenue_pct:.1f}% du CA")

In [ ]:
# Visualisation Pareto
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(1, len(customer_revenue) + 1) / len(customer_revenue) * 100
y = customer_revenue.cumsum() / customer_revenue.sum() * 100

ax.plot(x, y, 'b-', linewidth=2)
ax.axhline(80, color='red', linestyle='--', alpha=0.7, label='80% du CA')
ax.axvline(20, color='green', linestyle='--', alpha=0.7, label='20% des clients')
ax.fill_between(x, y, alpha=0.3)

ax.set_xlabel('% cumulé des clients (triés par revenue décroissant)')
ax.set_ylabel('% cumulé du CA')
ax.set_title('Courbe de Pareto - Concentration de valeur client')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "01_pareto_curve.png", dpi=200, bbox_inches="tight")
plt.show()

### INSIGHT PARETO

**~20% des clients génèrent ~X% du CA** → Forte concentration de valeur.

**Implication** : La segmentation RFM permettra d'identifier ces clients à haute valeur ("Champions") vs clients à risque.

---
## 4. RFM Exploratoire

**Objectif** : Calculer R/F/M provisoires et analyser leurs distributions

In [ ]:
# Date de référence = max + 1 jour
reference_date = df_valid['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Date de référence: {reference_date}")

In [ ]:
# Calcul RFM
rfm = df_valid.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'TotalAmount': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
print(f"Shape RFM: {rfm.shape}")
rfm.head(10)

In [ ]:
# Stats descriptives RFM
rfm[['Recency', 'Frequency', 'Monetary']].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
# Distributions RFM
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

# Row 1: Histogrammes
for ax, col, color in zip(axes[0], ['Recency', 'Frequency', 'Monetary'], ['#3498db', '#2ecc71', '#e74c3c']):
    ax.hist(rfm[col], bins=50, color=color, edgecolor='black', alpha=0.7)
    ax.axvline(rfm[col].median(), color='black', linestyle='--', label=f'Médiane: {rfm[col].median():.0f}')
    ax.set_xlabel(col)
    ax.set_ylabel('Clients')
    ax.set_title(f'Distribution {col}')
    ax.legend()

# Row 2: Boxplots
for ax, col, color in zip(axes[1], ['Recency', 'Frequency', 'Monetary'], ['#3498db', '#2ecc71', '#e74c3c']):
    bp = ax.boxplot(rfm[col], vert=True, patch_artist=True)
    bp['boxes'][0].set_facecolor(color)
    ax.set_ylabel(col)
    ax.set_title(f'Boxplot {col} (outliers visibles)')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "01_rfm_distributions.png", dpi=200, bbox_inches="tight")
plt.show()

In [ ]:
# Détection outliers (IQR)
def detect_outliers_iqr(series):
    Q1, Q3 = series.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = ((series < lower) | (series > upper)).sum()
    return outliers, outliers / len(series) * 100

print("Outliers (méthode IQR):")
for col in ['Recency', 'Frequency', 'Monetary']:
    n_out, pct_out = detect_outliers_iqr(rfm[col])
    print(f"  {col}: {n_out:,} outliers ({pct_out:.1f}%)")

In [ ]:
# Skewness (asymétrie)
print("Skewness (asymétrie):")
for col in ['Recency', 'Frequency', 'Monetary']:
    skew = rfm[col].skew()
    print(f"  {col}: {skew:.2f} {'(fortement asymétrique)' if abs(skew) > 1 else ''}")

In [ ]:
# Test log-transform sur Frequency et Monetary
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for ax, col in zip(axes, ['Frequency', 'Monetary']):
    log_values = np.log1p(rfm[col])  # log(1+x) pour gérer les 0
    ax.hist(log_values, bins=50, color='purple', edgecolor='black', alpha=0.7)
    ax.set_xlabel(f'log(1 + {col})')
    ax.set_ylabel('Clients')
    ax.set_title(f'Distribution log-transformée {col} (skew: {log_values.skew():.2f})')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "01_rfm_log_transform.png", dpi=200, bbox_inches="tight")
plt.show()

### DÉCISION TRANSFORMATIONS

| Feature | Skewness | Outliers | Décision |
|---------|----------|----------|----------|
| Recency | Modéré | ~X% | RobustScaler |
| Frequency | Fort | ~X% | Log-transform + RobustScaler |
| Monetary | Fort | ~X% | Log-transform + RobustScaler |

**Justification** : Distributions très skewed → log-transform réduit l'asymétrie, RobustScaler gère les outliers restants.

---
## 5. Validation préliminaire du scaling

**Objectif** : Comparer rapidement les approches de scaling

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# Préparation features
X_raw = rfm[['Recency', 'Frequency', 'Monetary']].values
X_log = np.column_stack([
    rfm['Recency'].values,
    np.log1p(rfm['Frequency'].values),
    np.log1p(rfm['Monetary'].values)
])

In [ ]:
# Comparaison scaling (K=4 fixe pour test rapide)
K_TEST = 4
results = []

scalers = {
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler(),
    'MinMaxScaler': MinMaxScaler()
}

for name, scaler in scalers.items():
    # Sans log
    X_scaled = scaler.fit_transform(X_raw)
    km = KMeans(n_clusters=K_TEST, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)
    results.append({'Méthode': name, 'Log': 'Non', 'Silhouette': sil})
    
    # Avec log
    X_scaled_log = scaler.fit_transform(X_log)
    labels_log = km.fit_predict(X_scaled_log)
    sil_log = silhouette_score(X_scaled_log, labels_log)
    results.append({'Méthode': name, 'Log': 'Oui', 'Silhouette': sil_log})

results_df = pd.DataFrame(results).pivot(index='Méthode', columns='Log', values='Silhouette')
results_df = results_df[['Non', 'Oui']]  # Réordonner colonnes
results_df

In [ ]:
# Visualisation comparaison
fig, ax = plt.subplots(figsize=(8, 4))
results_df.plot(kind='bar', ax=ax, color=['#e74c3c', '#2ecc71'])
ax.set_ylabel('Score Silhouette (K=4)')
ax.set_title('Comparaison Scaling : Impact du log-transform')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend(title='Log-transform')
ax.axhline(0, color='black', linewidth=0.5)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "01_scaling_comparison.png", dpi=200, bbox_inches="tight")
plt.show()

### DÉCISION SCALING

**Choix retenu** : `Log-transform (F, M)` + `RobustScaler`

**Justification** :
- Log-transform améliore le score silhouette
- RobustScaler résiste aux outliers résiduels
- Combinaison robuste pour données e-commerce (distributions skewed typiques)

---
## 6. Synthèse EDA → Décisions

### Règles de nettoyage

| # | Règle | Impact |
|---|-------|--------|
| 1 | Supprimer `CustomerID` null | -25% lignes |
| 2 | Exclure `Quantity ≤ 0` | -2% lignes |
| 3 | Exclure `UnitPrice ≤ 0` | -0.4% lignes |
| 4 | Exclure factures `C` (annulées) | Inclus dans #2 |

### Feature engineering

| # | Action |
|---|--------|
| 5 | Créer `TotalAmount = Quantity × UnitPrice` |
| 6 | Log-transform sur Frequency et Monetary |
| 7 | RobustScaler sur R, log(F), log(M) |

### Insights business

| Métrique | Valeur |
|----------|--------|
| Concentration Pareto | ~20% clients = ~X% CA |
| AOV | £X |
| Clients segmentables | ~4,300 |

In [ ]:
# Estimation finale
print("=== ESTIMATION POST-PREPROCESSING ===")
print(f"Lignes conservées: {len(df_valid):,} / {len(df):,} ({len(df_valid)/len(df)*100:.1f}%)")
print(f"Clients pour clustering: {rfm.shape[0]:,}")

---

**EDA VALIDÉ** → Prochaine étape : `02_preprocessing.ipynb`